In [ ]:
import pandas as pd
import numpy as np
import scipy.io as sio
import os
import re
import matplotlib.pyplot as plt
import source.transform_data as trs
from pathlib import Path

In [ ]:
path_to_data = Path('Data')

### Load data from csv-files:

In [ ]:
data_wide = pd.read_csv(path_to_data/'data_table_wide.csv', index_col=None)

In [ ]:
data_long = pd.read_csv(path_to_data/'data_table_long.csv', index_col=None)

In [ ]:
data_wide.head()

## Load data from .mat files and save them in csv files

### Get list of files to load: 

In [ ]:
path_to_data = Path('Data')

In [ ]:
path_to_data

In [ ]:
file_list = os.listdir(path_to_data)

In [ ]:
file_list = os.listdir(path_to_data)
file_list_mat = [x for x in  file_list if x[-4:] == '.mat']
len(file_list_mat)

### Sort file names in list alpha-numerically:

In [ ]:
file_list_mat.sort()
#file_list_mat

### Load all files into a dictionary: 

In [ ]:
#os.chdir('Data')

data_all = dict()

for file in file_list_mat:
    mat_contents = sio.loadmat(path_to_data/file)
    data_all[file] = mat_contents

In [ ]:
os.getcwd()

In [ ]:
data_all[file_list_mat[0]]['results'].dtype.descr[0][0]

In [ ]:
data_all[file_list_mat[0]]['results'].dtype.descr[1][0]

### Get the variables referring to days of measurement:

In [ ]:
nfoot_var = data_all[file_list_mat[0]]['results'].dtype.descr[0][0]
steps_tot_var = data_all[file_list_mat[0]]['results'].dtype.descr[1][0]

In [ ]:
round_counter = 1

for key, data_item in data_all.items():
    print(key)
    print(key.split('.'))
    print(key.split('.')[0])
    #print(data_item['results'].dtype.descr[0][0])
    nfoot_var_name = data_item['results'].dtype.descr[0][0]
    steps_tot_var_name = data_item['results'].dtype.descr[1][0]
    print(len(data_item['results'][0][0][0][0]))
    print(len(data_item['results'][0][0][1][0]))
    round_counter += 1
    if round_counter >= 3:
        break

In [ ]:
round_counter = 0
list_of_frames_days = []

for file_name, data_item in data_all.items():

    nfoot_var_name = data_item['results'].dtype.descr[0][0]
    steps_tot_var_name = data_item['results'].dtype.descr[1][0]
    nfoot_var = data_item['results'][0][0][0][0]
    steps_tot_var = data_item['results'][0][0][1][0]

    data_to_frame_days = {}

    data_to_frame_days['file_name'] = [file_name]*len(nfoot_var)

    name_parts = file_name.split('_')[:2]
    subject_name = '_'.join(name_parts)
    subject_names = [subject_name]*len(nfoot_var)
    data_to_frame_days['subject'] = subject_names

    data_to_frame_days[nfoot_var_name] = nfoot_var
    data_to_frame_days[steps_tot_var_name] = steps_tot_var

    data_frame_to_stack_days = pd.DataFrame(data_to_frame_days)
    
    list_of_frames_days.append(data_frame_to_stack_days)
data_days = pd.concat(list_of_frames_days)

In [ ]:
data_days

In [ ]:
data_days = trs.add_exp_phase_id(data_days, 'file_name')
data_days.head()

### Get the variables referring to files:

In [ ]:
round_counter = 0
list_of_frames_files = []

timestamps_indices = []
vars_same_length_checks = []

for file_name, data_item in data_all.items():

    nfoot_var_name = data_item['results'].dtype.descr[0][0]
    steps_tot_var_name = data_item['results'].dtype.descr[1][0]
    length_nfoot_var = len(data_item['results'][0][0][0][0])
    length_steps_tot_var = len(data_item['results'][0][0][1][0])

    if length_nfoot_var == length_steps_tot_var:
        number_of_days = length_nfoot_var
    else:
        print('Error: nfoot not equal to steps_tot!!')
        print(file_name)
        break

    # Get variable length and check if all variables (except nfoot and steps_tot) have the same length:
    var_name = data_item['results'].dtype.descr[2][0]    
    var_length = len(data_item['results'][0][0][2][0])
    num_elements = len(data_item['results'].dtype.descr)
    var_lengths = []
    for index in range(2, num_elements):
        var_name_i = data_item['results'].dtype.descr[index][0]
        var_length_i = len(data_item['results'][0][0][index][0])
        var_lengths.append(var_length_i)
        if var_length_i != var_length:
            print('Error: variables not the same length!!')
            print(filename)
            print(var_name)
            var_length = 'Difference in variable length!'

    all_vars_same_length = (len(set(var_lengths)) == 1)
    #print('\n')
    #print('All vars same length?')
    #print(all_vars_same_length)
    vars_same_length_checks.append(all_vars_same_length)
    if all_vars_same_length:
        number_of_walking_bouts = var_length
    else:
        print('Error variables not same length!')

    # Get gait_timestamps index to check if it is the same for all files, also get number
    # of walking bouts in the morning:
    num_elements = len(data_item['results'].dtype.descr)
    #variable_names = []
    for index in range(0, num_elements):
        variable_name = data_item['results'].dtype.descr[index][0]
        if variable_name == 'gait_timestamps':
            #print('\n Check if var name is gait_timestamps:')
            #print(variable_name)
            #print(index)
            time_index = index
            timestamps_indices.append(index)
            morning_walks = data_item['results'][0][0][index][0]/3600 < 4
            num_morning_walks = sum(morning_walks)
        #variable_names.append(variable_name)
    variable_of_interest = 'gait_timestamps'
    #variable_index_timestamps = [i for i, expr in enumerate(variable_names) if re.search(variable_of_interest, expr)][0]
    #print('\ncheck timestamp index:')
    #print(time_index == variable_index_timestamps)
    #timestamps_indices.append(variable_index_timestamps)
    
    #morning = data_item['results'][0][0][variable_index_timestamps][0]/3600 < 4

    data_to_frame_files = {}

    data_to_frame_files['file_name'] = [file_name]
    
    name_parts = file_name.split('_')[:2]
    subject_name = '_'.join(name_parts)
    data_to_frame_files['subject'] = [subject_name]
    
    
    data_to_frame_files['number_of_days'] = [length_nfoot_var]
    data_to_frame_files['number_of_walking_bouts'] = [number_of_walking_bouts]
    data_to_frame_files['number_of_morning_walking_bouts'] = num_morning_walks

    data_frame_to_stack_files = pd.DataFrame(data_to_frame_files)
    
    list_of_frames_files.append(data_frame_to_stack_files)

data_files = pd.concat(list_of_frames_files)


In [ ]:
print(len(set(timestamps_indices)))

print(len(vars_same_length_checks) == sum(vars_same_length_checks))

In [ ]:
data_files

In [ ]:
data_files = trs.add_exp_phase_id(data_files, 'subject')

In [ ]:
data_files

In [ ]:
data_files.head(3)

### Make table with variables referring to subjects:

In [ ]:
columns_to_select = ['subject', 'exp_phase_id', 'number_of_days', 
                     'number_of_walking_bouts', 'number_of_morning_walking_bouts']
data_subjects_c = data_files[columns_to_select]

In [ ]:
columns_combine = ['exp_phase_id']
values_combine = ['number_of_days', 
                     'number_of_walking_bouts', 
                   'number_of_morning_walking_bouts']

In [ ]:
data_subjects = data_subjects_c.pivot(index='subject', columns=columns_combine, 
                      values=values_combine)

In [ ]:
data_subjects.columns

In [ ]:
data_subjects.head()

In [ ]:
new_col_names = []
for col in data_subjects.columns:
    new_col_name = '_'.join(list(col))
    print(new_col_name)
    new_col_names.append(new_col_name)

In [ ]:
data_subjects.columns = new_col_names

In [ ]:
data_subjects.reset_index(inplace=True)

In [ ]:
data_subjects.head()

### Solution provided by Chatgpt:

In [ ]:
# Melt the DataFrame
melted = pd.melt(data_subjects_c, id_vars=['subject', 'exp_phase_id'], var_name='metric', value_name='value')

# Create a new column by concatenating the phase and metric
melted['new_column'] = melted['metric'] + '_' + melted['exp_phase_id']

# Pivot the DataFrame using the new column names
pivoted = melted.pivot(index='subject', columns='new_column', values='value')

# Flatten the columns
pivoted.columns.name = None
pivoted.reset_index(inplace=True)

# Display the resulting DataFrame
pivoted.head()

In [ ]:
#pivoted_co = pivoted[data_subjects.columns]

pivoted_co = pivoted.reindex(columns=data_subjects.columns)

In [ ]:
data_subjects.equals(pivoted_co)

### Compare data frames to data_wide and data_long to check consistency:

In [ ]:
data_wide.head()

In [ ]:
data_files.head()

In [ ]:
data_subjects.head()

In [ ]:
subject = 'Sub_30'
exp_phase = 'T3'
morning = 1

In [ ]:
s_bools = data_wide.subject == subject
exp_phase_bools = data_wide.exp_phase_id == exp_phase
morning_bools = data_wide.morning_afternoon == morning

dw_select = data_wide.loc[(s_bools & exp_phase_bools & morning_bools)]
dw_select.head()

In [ ]:
print(set(dw_select.subject))
print(set(dw_select.exp_phase_id))
print(set(dw_select.morning_afternoon))

In [ ]:
dw_select.shape

In [ ]:
s_bools = data_subjects.subject == subject

ds_select = data_subjects.loc[(s_bools)]
ds_select.head()

In [ ]:
ds_select.number_of_walking_bouts_T2 - ds_select.number_of_morning_walking_bouts_T2

In [ ]:
ds_select.number_of_walking_bouts_T3 - ds_select.number_of_morning_walking_bouts_T3

In [ ]:
ds_select.number_of_days_T3 + ds_select.number_of_days_T2

In [ ]:
s_bools = data_files.subject == subject
exp_phase_bools = data_files.exp_phase_id == exp_phase
df_select = data_files.loc[(s_bools&exp_phase_bools)]
df_select.head()

In [ ]:
df_select.number_of_walking_bouts

In [ ]:
df_select.number_of_morning_walking_bouts

In [ ]:
df_select.number_of_walking_bouts - df_select.number_of_morning_walking_bouts

In [ ]:
s_bools = data_days.subject == subject
exp_phase_bools = data_days.exp_phase_id == exp_phase
dd_select = data_days.loc[(s_bools & exp_phase_bools)]
dd_select.shape

In [ ]:
dd_select

### Save data frames:

In [ ]:
data_files.to_csv(path_to_data/'data_table_files.csv', sep=',', index=False)
data_days.to_csv(path_to_data/'data_table_days.csv', sep=',', index=False)
data_subjects.to_csv(path_to_data/'data_table_subjects.csv', sep=',', index=False)